### REINFORCE Function Optimization using RL algorithms
### Based on R.J. Williams and J. Ping (1991)
#### Problem Formulation:
Consider a a generate-and-test scenario for adaptive function optimization.
Follow Ackley using problems for which the optimum point is known and simply run the 
algorithm until either such a point is generated or some maximum computational effort
used is the number of function evaluations performed.
The broad goal of any adaptive sampling scheme is to try reshape the sampling distribution in ways 
that make it more likely to sample better points. Applying a RL algo to the weights can be viewed as a means of
doing just that when the sample points are generated by a network.

In [ ]:
## Notation  and Terminology
##
## n_I, external inputs from the environment
## n_O, output units which affect the environment
## n_H, hidden units
## x_n, the n_I tuple of external input signals to the network at a particular time
## y_n, n_O tuple of network output produced as a result
## y, n_O + n_H tuple
## x, (n_I + n_H + n_O) tuple obtained by concatenating x_N and y
## Typical element of x is x[j], which is either the output of the jth unit in the network, 
## or the value received on the jth input line
## Let W, (n_H + n_O) x (n_I + n_H + n_O) matrix denote the weight matrix for the network, with exactly one weight between each pair of units 
## and also form each input line to each unit
## The weight w[i,j] represents the weight on the connection to the ith unit from either the jth unit or the jth input line
## To accomodate a bias weight for each unit, we simply include among the n_I input lines one input whose value is always 1. 
## We adopt the convention that this  bias input has an index of 0, so that w[i,0] represents the bias weight for the ith unit
## In this case, no external input (except we still need the bias input)
## Each time the network computes an  output vector y_n is assumed to represent a single trial point 
## for the function to be optimized
## J is the function to be optimized
##
## We Use Logistic Units throughout



In [46]:
# Utilities, functions

import math
import numpy as np

def sigmoid(x):
    """
    Compute sigmoid of x.
    Arguments:
    x -- A scalar
    Return:
    s -- sigmoid(x)
    """
    s = 1/(1+math.exp(-x))
    
    return s


def one_max(u):
    """
    Compute One-Max function.
    Arguments:
    u: n-dimensional bit vector(u_1, u_2, ..., u_n)
    Return: 
    J(u) = 10*(numbers of 1s in u)
    
    """
    J = 10*np.sum(u)
    return J

def two_max(u):
    """
    Compute Two-Max function.
    Arguments:
    u: n-dimensional bit vector(u_1, u_2, ..., u_n)
    Return: 
    J(u) = 18*|18*(numbers of 1s) - 8*n|
    """
    J = abs(18*sum(u)-8*len(u))
    return J

def porcupine(u):
    """
    Compute the Porcupine function.
    Arguments:
    u: n-dimensional bit vector(u_1, u_2, ..., u_n)
    Return:
    n_1 = number of 1's in u
    n_0 = number of o's in u
    J(u) = 10n_1 - 15(n_0 mod 2)
    
    """
    n_1 = sum(u)
    n_0 = len(u) - n_1
    J = 10*n_1 - 15*(n_0 % 2)
    return J

def plateaus(u):
    """
    Compute the Plateaus Function.
    Arguments:
    u: n-dimensional bit vector(u_1, u_2, ..., u_n)
    n must be a multiple of 4
    Return:
    J(u) = Plateaus as defined eqs (9) and (10) in Williams&Peng 1991
    """ 
    J = 0
    J_k = 1
    n = len(u)
    assert (n % 4 == 0), 'n must be a multiple of 4.'
    for k in range(4):
        for i in range(int(k * n/4+1), int((k+1) * n/4)):
                J_k = J_k * 2.5 * n * u[i]
        J = J + J_k           
    return J

In [47]:
sigmoid(-1)
u = np.array([0, 1, 0, 1, 1, 1, 0, 0, 0 ])
one_max(u)
abs(-10)
u2 = np.array([0,0,0,0])
two_max(u2)
u3=np.array([1,1,1,1])
two_max(u3)
15%4
porcupine(u3)
porcupine(u2)
u4 = np.array([0, 1, 0, 1])
porcupine(u4)
plateaus(u4)


4

In [48]:
# 4.1 Team Networks
# Initialize parameters
n = 20
w_0 = np.zeros(n)
p_i = np.zeros(n)
y_0 = np.zeros(n)


(1, 1)